# **Day 11: Cosmic Expansion**

# Setup
The cells below will set up the rest of the notebook. 

I'll start by configuring my kernel:

In [1]:
# Changing the current working directory
%cd ..

# Enabling the autoreload extension
%load_ext autoreload
%autoreload 2

d:\data\programming\advent-of-code-2023


Now, I'm going to import some libraries:

In [2]:
# Import statements
import pandas as pd
import re
from tqdm import tqdm
from concurrent.futures import ThreadPoolExecutor

Finally, I'll load in the data for this puzzle. 

In [3]:
# Load in the data for the puzzle
day = 11
input_data_path = f"data/input-files/day-{day:02d}-input.txt"
example_data_path = f"data/example-input/day-{day:02d}-example.txt"
with open(input_data_path, "r") as txt_file:
    input_data = txt_file.readlines()

# Expanding Space
One of the first things I need to do: figure out how to "expand space". Here's the part of the puzzle that mentions it: 

```
Due to something involving gravitational effects, only some space expands. In fact, the result is that any rows or columns that contain no galaxies should all actually be twice as big.
```

This shouldn't be too hard. Just pull it into a grid structure and then transform the grid. 

Before I do that, though, I want to quickly define a `Grid` class. I want to try and make some functions that are generally usable throughout a couple of puzzles - not sure I'll perfect it here, but this one would probably be nice to have. 

In [4]:
# Create a class for grids
class Grid:
    def __repr__(self):
        """
        Create a string representation of the grid
        """
        all_rows = ""
        for row in self.internal_grid:
            all_rows += "".join(row) + "\n"
        return all_rows

    def __str__(self):
        return self.__repr__()

    def __init__(self, internal_grid):
        """
        The initialization method
        """

        self.internal_grid = internal_grid
        self.dim = (len(internal_grid), len(internal_grid[0]))

    def access(self, row_idx, col_idx):
        """
        Access a coordinate in the grid
        """

        try:
            accessed_value = self.internal_grid[row_idx][col_idx]
        except IndexError as e:
            raise Exception(f"ERROR ('{e}'): Attempted to access an index that did not exist.")
        except Exception as e:
            raise e

        # If we succeeded, return the accessed value
        return accessed_value

With that class in hand, we can go about expanding the galaxy. I'll start by figuring out which rows / columns need to be expanded. 

In [5]:
# Create a space_grid from the input_data
original_space_array = [[char for char in line if char != "\n"] for line in input_data]
original_space_grid = Grid(original_space_array)

# Determine the coordinates of the galaxies
original_galaxy_coordinates = []
for x_coord in range(original_space_grid.dim[0]):
    for y_coord in range(original_space_grid.dim[1]):
        if original_space_grid.access(x_coord, y_coord) == "#":
            original_galaxy_coordinates.append((x_coord, y_coord))

# Determine which rows and columns don't have any galaxies
rows_with_no_galaxies = [
    row_idx
    for row_idx in range(original_space_grid.dim[0])
    if row_idx not in [coord[0] for coord in original_galaxy_coordinates]
]
cols_with_no_galaxies = [
    col_idx
    for col_idx in range(original_space_grid.dim[1])
    if col_idx not in [coord[1] for coord in original_galaxy_coordinates]
]

original_space_grid

.....................................................................................................#..................#......#............
...................#.........#...............................................#.........#...................#............................#...
............#.............................................#.................................................................................
.......#...............#...........#.............#.................................#............................#..................#........
..................................................................#........................................................................#
.........................................................................................#.......#..................#.......................
....#...........#.............................................#........................................#....................................
.............

Now, I'll make the expanded grid:

In [6]:
expansion_multiplier = 3

# Create a new expanded space array, first starting with expanding row-wise
temp_expanded_space_array = []
for row_idx in range(original_space_grid.dim[0]):
    cur_row = [
        original_space_grid.access(row_idx, col_idx)
        for col_idx in range(original_space_grid.dim[1])
    ]
    temp_expanded_space_array.append(cur_row)

    # If this row doesn't have a galaxy, add it again
    if row_idx in rows_with_no_galaxies:
        for _ in range(expansion_multiplier-1):
            temp_expanded_space_array.append(cur_row)
temp_expanded_space_grid = Grid(temp_expanded_space_array)

# Now, we'll need to add the extra columns. This is slightly trickier with basic array manipulation
n_new_cols = temp_expanded_space_grid.dim[1] + len(rows_with_no_galaxies) * (expansion_multiplier-1)
expanded_space_array = [[] for _ in range(n_new_cols)]
for old_col_idx in range(original_space_grid.dim[1]):
    cur_col = [
        temp_expanded_space_grid.access(row_idx, old_col_idx)
        for row_idx in range(temp_expanded_space_grid.dim[0])
    ]
    for row_idx, val in enumerate(cur_col):
        expanded_space_array[row_idx].append(val)
    if old_col_idx in cols_with_no_galaxies:
        for row_idx, val in enumerate(cur_col):
            for _ in range(expansion_multiplier-1):
                expanded_space_array[row_idx].append(val)
expanded_space_grid = Grid(expanded_space_array)

# Determine the coordinates of the galaxies in the expanded space grid
expanded_space_galaxy_coordinates = []
for x_coord in range(expanded_space_grid.dim[0]):
    for y_coord in range(expanded_space_grid.dim[1]):
        if expanded_space_grid.access(x_coord, y_coord) == "#":
            expanded_space_galaxy_coordinates.append((x_coord, y_coord))

Now that I've created an expanded grid, I can go about determining the pairwise distances between each of the galaxies. 

In [7]:
# Create a DataFrame with information about the galaxies
expanded_space_galaxies_df = pd.DataFrame(
    [
        {"galaxy_num": idx, "coords": coords}
        for idx, coords in enumerate(expanded_space_galaxy_coordinates)
    ]
)


def manhattan_distance(point_1, point_2):
    """
    This method calculates the Manhattan Distance between two points on a grid.
    """
    # Unpack the inputs
    x_1, y_1 = point_1
    x_2, y_2 = point_2

    # Return the Manhattan Distance
    return abs(x_2 - x_1) + abs(y_2 - y_1)


def calculate_galaxy_pairwise_distances(galaxy_num, other_galaxy_num, galaxies_df):
    """
    This is a helper method for parallelization.
    """
    # Calculate the pairwise distance
    galaxy_coords = (
        galaxies_df.query("galaxy_num==@galaxy_num").iloc[0].coords
    )
    other_galaxy_coords = (
        galaxies_df.query("galaxy_num==@other_galaxy_num").iloc[0].coords
    )

    return manhattan_distance(galaxy_coords, other_galaxy_coords)

In [ ]:
# Figure out the pairwise distances between each galaxy in parallel
pairwise_distance_dict = {}
futures = {}
with ThreadPoolExecutor(max_workers=32) as executor:
    # Submit all of the futures
    for galaxy_num in tqdm(expanded_space_galaxies_df["galaxy_num"].unique()):
        for other_galaxy_num in [
            num
            for num in expanded_space_galaxies_df["galaxy_num"].unique()
            if num != galaxy_num
        ]:
            # Only calculate the distance if we haven't already done it
            galaxy_num_pair = (
                min([galaxy_num, other_galaxy_num]),
                max([galaxy_num, other_galaxy_num]),
            )
            if galaxy_num_pair not in futures:
                futures[galaxy_num_pair] = executor.submit(
                    calculate_galaxy_pairwise_distances, galaxy_num, other_galaxy_num,
                    expanded_space_galaxies_df
                )

    # Collect all of the futures
    for galaxy_num_pair, future in tqdm(list(futures.items())):
        pairwise_distance_dict[galaxy_num_pair] = future.result()

With these inputs in hand, I'll calculate their sum. 

In [ ]:
# Calculate and print the sum of the paths 
dist_sum = sum([distance for galaxy_num_pair, distance in pairwise_distance_dict.items()])
print(f"The sum of the shortest paths between each galaxy is '{dist_sum}'")

# Part 2: Massive Space Expansion
The big twist in this part: space is expanding a ton. 

I should've done this at the beginning, but: it's easy to define an expansion projection instead of recreating the grid 🙃 Here's the 2:30am solution:

In [8]:
def get_projected_coordinates(
    space_grid,
    expansion_amt,
    target_row_num,
    target_col_num,
    rows_with_no_galaxies=None,
    cols_with_no_galaxies=None,
):
    """
    This method will expand the grid by the expansion_amt, and then project the
    x and y coordinates onto this new grid.
    """

    if rows_with_no_galaxies is None or cols_with_no_galaxies is None:
        # Determine the coordinates of the galaxies
        original_galaxy_coordinates = []
        for x_coord in range(space_grid.dim[0]):
            for y_coord in range(space_grid.dim[1]):
                if space_grid.access(x_coord, y_coord) == "#":
                    original_galaxy_coordinates.append((x_coord, y_coord))

    # Determine which rows and columns don't have any galaxies
    if rows_with_no_galaxies is None:
        rows_with_no_galaxies = [
            row_idx
            for row_idx in range(space_grid.dim[0])
            if row_idx not in [coord[0] for coord in original_galaxy_coordinates]
        ]
    if cols_with_no_galaxies is None:
        cols_with_no_galaxies = [
            col_idx
            for col_idx in range(space_grid.dim[1])
            if col_idx not in [coord[1] for coord in original_galaxy_coordinates]
        ]

    # Determine the projected x coordinate
    n_galaxiless_rows = len(
        [row_num for row_num in rows_with_no_galaxies if row_num < target_row_num]
    )
    projected_x = target_row_num + (n_galaxiless_rows * (expansion_amt-1))
    
    # Determine the projected y coordinate
    n_galaxiless_cols = len(
        [col_num for col_num in cols_with_no_galaxies if col_num < target_col_num]
    )
    projected_y = target_col_num + (n_galaxiless_cols * (expansion_amt-1))
    
    # Return the projected coordinates
    return (projected_x, projected_y)

With this method, I can project each of the coordinates:

In [12]:
projection_amt = 1000000

projected_expansion_space_galaxies_df = pd.DataFrame.from_records([
    {
        "galaxy_num": galaxy_num,
        "original_coords": galaxy_coords,
        f"coords": get_projected_coordinates(
            space_grid=original_space_grid,
            expansion_amt= projection_amt,
            target_row_num=galaxy_coords[0],
            target_col_num=galaxy_coords[1],
            rows_with_no_galaxies=rows_with_no_galaxies,
            cols_with_no_galaxies=cols_with_no_galaxies
        ) 
    } for galaxy_num, galaxy_coords in enumerate(original_galaxy_coordinates)
])

Now, with this DataFrame, I should be able to just re-run my original parallelized distance calculation: 

In [13]:
# Figure out the pairwise distances between each galaxy in parallel
pairwise_distance_dict = {}
futures = {}
with ThreadPoolExecutor(max_workers=32) as executor:
    # Submit all of the futures
    for galaxy_num in tqdm(
        projected_expansion_space_galaxies_df["galaxy_num"].unique()
    ):
        for other_galaxy_num in [
            num
            for num in projected_expansion_space_galaxies_df["galaxy_num"].unique()
            if num != galaxy_num
        ]:
            # Only calculate the distance if we haven't already done it
            galaxy_num_pair = (
                min([galaxy_num, other_galaxy_num]),
                max([galaxy_num, other_galaxy_num]),
            )
            if galaxy_num_pair not in futures:
                futures[galaxy_num_pair] = executor.submit(
                    calculate_galaxy_pairwise_distances,
                    galaxy_num,
                    other_galaxy_num,
                    projected_expansion_space_galaxies_df,
                )

    # Collect all of the futures
    for galaxy_num_pair, future in tqdm(list(futures.items())):
        pairwise_distance_dict[galaxy_num_pair] = future.result()

100%|██████████| 105111/105111 [00:00<00:00, 841427.24it/s]


With these inputs in hand, I'll calculate their sum. 

In [14]:
# Calculate and print the sum of the paths 
dist_sum = sum([distance for galaxy_num_pair, distance in pairwise_distance_dict.items()])
print(f"The sum of the shortest paths between each galaxy is '{dist_sum}'")

The sum of the shortest paths between each galaxy is '707505470642'
